In [1]:
import datetime
from pyspark import SparkConf, SparkContext, HiveContext
sc.stop()
conf = SparkConf().set("spark.executor.instances", 1).set("spark.driver.maxResultSize", "1g")
sc = SparkContext(conf=conf)
hc = HiveContext(sc)
#sc.setCheckpointDir('/user/kposminin/checkpointdir/')

In [4]:
create_result_table_query = '''

create table user_kposminin.cold_liru_results_20170126 as
select
  to_date(create_dt) as call_ymd,
  b.phone_mobile,
  nvl(s.score,-20) as score,
  if(status = 'В работе',1,0) as in_work,
  if(a.financial_application_rk is not Null, 1, 0) as full_app,
  if(a.decision_dt is not Null, 1, 0) as considered,
  if(a.decision_approve_dt is not Null, 1, 0) as approve,
  if(a.utilization_dt is not Null, 1, 0) as utilization
  
from prod_emart.short_applications_current b 
  left join prod_emart.financial_account_application a on b.financial_application_rk = a.financial_application_rk
  left join (select phone_num, score from prod_lookalike.phone_x_segment where segment_nm = 'la_apppr_ccall' and ymd between '2017-01-22' and '2017-01-24') s on substr(s.phone_num,3,20) = substr(b.phone_mobile,2,20)
where (not b.status in ('Дубль', 'Черный список'))
  and b.utm_campaign = 'cold_liru'
  and create_dt >= '2017-01-25' and create_dt < '2017-01-27'

'''
# Запрос выполнялся в хайве. При выполнении из спарк надо смотреть на корректность передачи кириллицы.

df = hc.sql('select * from user_kposminin.cold_liru_results_20170126').toPandas()
df.head()

,call_ymd,phone_mobile,score,in_work,full_app,considered,approve,utilization
0,2017-01-25,89633122223,-4.978807,1,0,0,0,0
1,2017-01-25,89619153434,-3.949567,0,0,0,0,0
2,2017-01-26,89504957407,-4.393766,1,0,0,0,0
3,2017-01-26,89612095505,-4.393766,1,0,0,0,0
4,2017-01-26,89202445024,-3.764830,1,0,0,0,0


In [43]:
#df[df.in_work==1].full_app.sum()
data = sorted(df[df.in_work==0][['score','approve']].values.tolist(), reverse = True)


In [44]:
from sklearn.metrics import roc_auc_score
   
print('Max score AUC ROC: {}'.format(roc_auc_score(y_true = [e[1] for e in data],y_score = [e[0] for e in data])))

Max score AUC ROC: 0.467465366927


In [45]:
def nvl(a,b):
    if a:
        return a
    else:
        return b
    
print('\nLift table for ph_stand_la\n\n{:<10}\t{:<10}\t{:<10}\t{:<10}\t{:<10}\t{:<10}\t{:<10}'.format(
        'q','pos_cnt','pos%','cnt','pos/cnt','lift','score'))
pos_tot = sum([e[1] for e in data])
pos_share_tot = float(pos_tot) / len(data)
for q in [0.99,0.98,0.95,0.9,0.8,0.7,0.6,0.5,0.4,0.3,0]:
    pos_cnt = sum([e[1] for e in data[:int((1-q)*len(data))]])
    cnt = int((1-q)*len(data))
    pos_share = float(pos_cnt)/cnt
    print('{:<10}\t{:<10}\t{:<10.2%}\t{:<10}\t{:<10.5%}\t{:<10.2f}\t{:<10.5f}'.format(q,
                                                                                      pos_cnt,
                                                                                      float(pos_cnt)/pos_tot,
                                                                                      cnt,pos_share,
                                                                                      pos_share/pos_share_tot,
                                                                                      nvl(data[int((1-q)*len(data))-1][0],-20)
                                                                                     ))


Lift table for ph_stand_la

q         	pos_cnt   	pos%      	cnt       	pos/cnt   	lift      	score     
0.99      	0.0       	0.00%     	31        	0.00000%  	0.00      	-3.74813  
0.98      	1.0       	2.17%     	62        	1.61290%  	1.10      	-3.76483  
0.95      	1.0       	2.17%     	156       	0.64103%  	0.44      	-3.76483  
0.9       	5.0       	10.87%    	312       	1.60256%  	1.09      	-4.13541  
0.8       	10.0      	21.74%    	625       	1.60000%  	1.09      	-4.39377  
0.7       	12.0      	26.09%    	938       	1.27932%  	0.87      	-4.95734  
0.6       	18.0      	39.13%    	1251      	1.43885%  	0.98      	-5.23706  
0.5       	23.0      	50.00%    	1564      	1.47059%  	1.00      	-5.47575  
0.4       	25.0      	54.35%    	1876      	1.33262%  	0.91      	-5.65396  
0.3       	29.0      	63.04%    	2189      	1.32481%  	0.90      	-5.76780  
0         	46.0      	100.00%   	3128      	1.47059%  	1.00      	-5.99839  


### Вывод: на прозвоне зависимость от скора не прослеживается по всем переменным: approve, full_app, in_work. 
### Причины: либо из-за однородной структуры выборки (верхняя 0.03% квантиль), либо в связи с неправильной работой классификатора.